In [16]:
pip install geopy


   ---------------------------------------- 0.0/125.4 kB ? eta -:--:--
   --- ------------------------------------ 10.2/125.4 kB ? eta -:--:--
   ---------------------------------------  122.9/125.4 kB 1.8 MB/s eta 0:00:01
   ---------------------------------------- 125.4/125.4 kB 1.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/40.3 kB ? eta -:--:--
   ---------------------------------------- 40.3/40.3 kB ? eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\spenc\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
%pip install requests

   ---------------------------------------- 0.0/64.9 kB ? eta -:--:--
   ---------------------------------------- 64.9/64.9 kB 1.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/167.3 kB ? eta -:--:--
   ---------------------------------------- 167.3/167.3 kB 9.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/99.9 kB ? eta -:--:--
   ---------------------------------------- 99.9/99.9 kB ? eta 0:00:00
   ---------------------------------------- 0.0/70.4 kB ? eta -:--:--
   ---------------------------------------- 70.4/70.4 kB ? eta 0:00:00
   ---------------------------------------- 0.0/126.3 kB ? eta -:--:--
   ---------------------------------------- 126.3/126.3 kB ? eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\spenc\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [20]:
import requests
import pandas as pd
import time
from geopy.geocoders import Nominatim

# Your API key
api_key = 'WTCKQMFXBWJSST6TGTZCVZF5Y' #spencerhamilton60@gmail.com
api_key = 'HR23S3BURZPNU7DBHQA9GAYZY' #rhamilt4@byu.edu
api_key = '3N2JVBHZHRS8SS4WVVA2DN7LW' #fake email 1
api_key = 'JAQ9AZB224ATD3UZ9YLR58USU' #fake email 2

# Base URL for VisualCrossingWeather API
base_url = 'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/'

# Load your dataset
df = pd.read_csv("tester_yeah_with_weather.csv")





In [21]:
first_row = df.head(1)  # Get the first row as a DataFrame
df.columns


Index(['Incident Date', 'Park Name', 'Cause of Death',
       'Cause of Death Group \n(Used in the NPS Mortality Dashboard) ',
       'Intent', 'Outcome', 'Sex', 'Age Range', 'Activity', 'temperature',
       'precipitation', 'humidity', 'date', 'latlong', 'latitude',
       'longitude'],
      dtype='object')

In [22]:
#change the date info: 
# Convert 'date' column from m/d/y to yyyy-MM-dd
df['date'] = pd.to_datetime(df['Incident Date'], format='%m/%d/%Y').dt.strftime('%Y-%m-%d')

# Check the first row to confirm the format
print(df.head(1))


  Incident Date                             Park Name Cause of Death  \
0      1/1/2007  Glen Canyon National Recreation Area   Undetermined   

  Cause of Death Group \n(Used in the NPS Mortality Dashboard)         Intent  \
0                                       Undetermined             Undetermined   

        Outcome   Sex Age Range      Activity  temperature  precipitation  \
0  Fatal injury  Male       65+  Not Reported          NaN            NaN   

   humidity        date                                latlong   latitude  \
0       NaN  2007-01-01  ('37.6580958', '-111.09320889669256')  37.658096   

    longitude  
0 -111.093209  


In [10]:
#get the unique places
my_unique = df['Park Name']

In [11]:
print(my_unique)

0              Glen Canyon National Recreation Area
1              Golden Gate National Recreation Area
2              Golden Gate National Recreation Area
3                             Natchez Trace Parkway
4                             Natchez Trace Parkway
                           ...                     
4630          Valley Forge National Historical Park
4631    Delaware Water Gap National Recreation Area
4632                  Point Reyes National Seashore
4633                     Grand Canyon National Park
4634    Delaware Water Gap National Recreation Area
Name: Park Name, Length: 4635, dtype: object


In [15]:
#first I need to get the park names into real addresses:



import requests
import time

def get_park_address(park_name):
    url = f'https://nominatim.openstreetmap.org/search?q={park_name}&format=json&limit=1'
    headers = {
        'User-Agent': 'sphamiltronics668 (spencerhamilton60@gmailcom)'  # Replace with your app name and contact info
    }
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        data = response.json()
        
        if data:
            park_info = data[0]
            return {
                'display_name': park_info['display_name'],
                'lat': park_info['lat'],
                'lon': park_info['lon']
            }
        else:
            print(f"Could not find address for {park_name}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Error fetching address for {park_name}: {e}")
        return None


# Step 1: Extract unique park names from your DataFrame
unique_parks = df['Outcome'].unique()  # Replace 'park_name' with the correct column name in your DataFrame

# Step 2: Fetch latitude and longitude for each unique park name
latlong_dict = {}
for park in unique_parks:
    address_info = get_park_address(park)  # Use the get_park_address function we defined earlier
    if address_info:
        latlong_dict[park] = (address_info['lat'], address_info['lon'])
    else:
        latlong_dict[park] = (None, None)  # In case address isn't found



Could not find address for National Capital Parks-East
Could not find address for Colonial National Historical Park
Could not find address for Clara Barton National Historic Site
Could not find address for President's Park (White House)
Could not find address for Not Reported
Could not find address for Yorktown Battlefield Part of Colonial National Historical Park


KeyError: 'park_name'

In [9]:
# Step 3: Map the latlong info to the original DataFrame and add a 'latlong' column
df['latlong'] = df['Park Name'].map(latlong_dict)

# The DataFrame now has a new 'latlong' column with tuples (lat, lon) for each park
print(df.head())  # Preview the updated DataFrame


NameError: name 'latlong_dict' is not defined

In [23]:
# Step 4: Count how many rows have valid latlong values (not equal to (None, None))
valid_latlong_count = df[df['latlong'] != (None, None)].shape[0]

# Display the result
print(f"Number of parks with valid latlongs: {valid_latlong_count}")
print(f"which is pretty good bc there a total of {df.shape} observations")


Number of parks with valid latlongs: 1
which is pretty good bc there a total of (1, 16) observations


In [8]:
#save the latlong version
# Save the updated DataFrame to a CSV file
df.to_csv('national_parks_with_coordinates.csv', index=False)

print("Dataset saved as national_parks_with_coordinates.csv")


Dataset saved as national_parks_with_coordinates.csv


In [24]:
# Function to get weather data for a specific location and date
def get_weather_data(location, date):
    url = f'{base_url}{location}/{date}?key={api_key}'
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data.get('currentConditions', {})
    else:
        print(f"Error fetching data for {location} on {date}: {response.status_code}")
        return {}

In [30]:
for i, row in df.head(1).iterrows():  # Removed head(1) to loop through the entire DataFrame
    location = f"{row['latlong'][0]},{row['latlong'][1]}"
    print(location)

37.6580958,-111.09320889669256


In [34]:
# Function to safely extract latitude and longitude
def extract_lat_long(latlong):
    if latlong == (None, None):  # Check for (None, None)
        return None, None
    elif isinstance(latlong, tuple) and len(latlong) == 2:
        return float(latlong[0]), float(latlong[1])
    else:
        return None, None  # Return None for invalid data

# Apply the function to create new columns
df['latitude'], df['longitude'] = zip(*df['latlong'].apply(extract_lat_long))

# Check the updated DataFrame
print(df[['latlong', 'latitude', 'longitude']].head())


df.to_csv('national_parks_with_float_coors.csv', index=False)


                             latlong   latitude   longitude
0  (37.6580958, -111.09320889669256)  37.658096 -111.093209
1         (37.8499275, -122.5177518)  37.849927 -122.517752
2         (37.8499275, -122.5177518)  37.849927 -122.517752
3          (32.0071549, -90.8585141)  32.007155  -90.858514
4          (32.0071549, -90.8585141)  32.007155  -90.858514


In [36]:
#This is the one I'm using ___


# Loop through your dataset
for i, row in df.head(1).iterrows():  # Removed head(1) to loop through the entire DataFrame
    # Format the location as a string with six decimal places
    location = f"{row['latitude']:.6f},{row['longitude']:.6f}"  # Limits to six decimal places

    date = row['date']  # Ensure this is in the correct format (yyyy-MM-dd)
    
    if row['latlong'] == (None, None):  # Check for valid latlong
        continue
    
    # Request weather data
    weather_data = get_weather_data(location, date)
    
    # Append data to the respective columns
    if weather_data:
        df.at[i, 'temperature'] = weather_data.get('temp', None)
        df.at[i, 'precipitation'] = weather_data.get('precip', None)
        df.at[i, 'humidity'] = weather_data.get('humidity', None)
    
    # To avoid hitting the API rate limit, sleep between requests (adjust as needed)
    time.sleep(1)

# Save the updated dataset with weather data
df.to_csv('dataset_with_weather.csv', index=False)


In [12]:
df.head(1)

,Incident Date,Park Name,Cause of Death,Cause of Death Group \n(Used in the NPS Mortality Dashboard),Intent,Outcome,Sex,Age Range,Activity
0,1/1/2007,Glen Canyon National Recreation Area,Undetermined,Undetermined,Undetermined,Fatal injury,Male,65+,Not Reported


In [39]:
unique_parks = df['Outcome'].unique() 
print(unique_parks)

['Fatal injury' 'Fatal Injury']


In [25]:
# new Real??
def get_weather_data(location, date):
    url = f'{base_url}{location}/{date}?key={api_key}'
    response = requests.get(url)
    
    if response.status_code == 200:
        # Extract the relevant data
        data = response.json()
        
        # Here we assume that data contains hourly weather info, adapt based on actual structure
        hourly_data = data.get('hours', [])
        filtered_data = []
        
        for hour in hourly_data:
            filtered_data.append({
                'datetime': hour.get('datetime'),
                'temp': hour.get('temp'),
                'humidity': hour.get('humidity'),
                'precip': hour.get('precip'),
                'conditions': hour.get('conditions'),
            })
        
        return filtered_data
    else:
        print(f"Error fetching data for {location} on {date}: {response.status_code}")
        return []

# Example usage in your loop
for i, row in tester_guy.iterrows():
    latitude = row['latitude']
    longitude = row['longitude']
    date = row['date']

    if (latitude is None) or (longitude is None):
        continue

    # Format location
    location = f"{latitude:.6f},{longitude:.6f}"
    
    # Request weather data
    weather_data = get_weather_data(location, date)

    # Append filtered data to the DataFrame
    for hour_data in weather_data:
        df.at[i, 'datetime'] = hour_data['datetime']
        df.at[i, 'temperature'] = hour_data['temp']
        df.at[i, 'humidity'] = hour_data['humidity']
        df.at[i, 'precipitation'] = hour_data['precip']
        df.at[i, 'conditions'] = hour_data['conditions']
    
    # Sleep to avoid hitting the API rate limit
    time.sleep(1)

# Save the updated dataset with relevant weather data
tester_guy.to_csv('tester_yeah_with_weather.csv', index=False)


NameError: name 'tester_guy' is not defined

In [26]:
test_park = "Yosemite National Park"
test_date = "2021-07-15"

url = f'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/37.7213,-119.626/{test_date}?key={api_key}'
response = requests.get(url)
print(response.json())


{'queryCost': 24, 'latitude': 37.7213, 'longitude': -119.626, 'resolvedAddress': '37.7213,-119.626', 'address': '37.7213,-119.626', 'timezone': 'America/Los_Angeles', 'tzoffset': -7.0, 'days': [{'datetime': '2021-07-15', 'datetimeEpoch': 1626332400, 'tempmax': 87.6, 'tempmin': 56.5, 'temp': 71.6, 'feelslikemax': 84.0, 'feelslikemin': 56.5, 'feelslike': 70.6, 'dew': 29.0, 'humidity': 24.4, 'precip': 0.0, 'precipprob': 0.0, 'precipcover': 0.0, 'preciptype': None, 'snow': 0.0, 'snowdepth': 0.0, 'windgust': 28.6, 'windspeed': 7.6, 'winddir': 216.4, 'pressure': 1014.4, 'cloudcover': 0.3, 'visibility': 9.6, 'solarradiation': 370.5, 'solarenergy': 32.1, 'uvindex': 10.0, 'sunrise': '05:49:06', 'sunriseEpoch': 1626353346, 'sunset': '20:19:41', 'sunsetEpoch': 1626405581, 'moonphase': 0.2, 'conditions': 'Clear', 'description': 'Clear conditions throughout the day.', 'icon': 'clear-day', 'stations': ['72065200433', '99999953150', '72389403181', 'KBAN', '72061500206', 'CQ160', 'EPWC1', 'KMMH'], 'so

In [5]:
import requests

test_park = "Yosemite National Park"
test_date = "2021-07-15"
api_key = 'HR23S3BURZPNU7DBHQA9GAYZY' #rhamilt4@byu.edu

url = f'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/37.7213,-119.626/{test_date}?key={api_key}&include=hours'
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()
    # Filter to get only the 12 p.m. data
    noon_data = next((hour for hour in data['days'][0]['hours'] if hour['datetime'] == '12:00:00'), None)
    print(noon_data)
else:
    print("Failed to retrieve data:", response.status_code)


{'datetime': '12:00:00', 'datetimeEpoch': 1626375600, 'temp': 85.0, 'feelslike': 81.9, 'humidity': 8.09, 'dew': 17.4, 'precip': 0.0, 'precipprob': 0.0, 'snow': 0.0, 'snowdepth': 0.0, 'preciptype': None, 'windgust': 20.8, 'windspeed': 5.3, 'winddir': 220.0, 'pressure': 1011.9, 'visibility': 9.1, 'cloudcover': 0.0, 'solarradiation': 964.0, 'solarenergy': 3.5, 'uvindex': 10.0, 'conditions': 'Clear', 'icon': 'clear-day', 'stations': ['72065200433', '99999953150', 'KBAN', 'CQ160', '72061500206'], 'source': 'obs'}


In [44]:
weather_data = get_weather_data("37.7213,-119.626", "2021-06-15")

In [45]:
print(weather_data)

[]


In [40]:
import requests
import pandas as pd
import time

# Define the base URL for the VisualCrossingWeather API
base_url = 'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/'

# Function to get full daily weather data
def get_weather_data(location, date, api_key):
    # Update the URL to include the 'include' parameter to fetch only daily data
    url = f'{base_url}{location}/{date}?key={api_key}&include=days'
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        return data['days'][0]  # This now includes only the daily data
    elif response.status_code in [403, 429]:  # Common codes for invalid key or rate limit
        print(f"API key issue or rate limit reached: {response.status_code}")
        return "STOP"
    else:
        print(f"Error fetching data for {location} on {date}: {response.status_code}")
        return None



In [27]:
import pandas as pd
# Add a new column to store the raw JSON weather data
# Load your dataset
df = pd.read_csv("weather__ed.csv")
#df['Weather'] = None



In [28]:
dfs = df.head(1)
api_key = 'HR23S3BURZPNU7DBHQA9GAYZY' #rhamilt4@byu.edu

print(dfs)

  Incident Date                             Park Name Cause of Death  \
0      1/1/2007  Glen Canyon National Recreation Area   Undetermined   

  Cause of Death Group \r\n(Used in the NPS Mortality Dashboard)   \
0                                       Undetermined                

         Intent       Outcome   Sex Age Range      Activity  temperature  \
0  Undetermined  Fatal injury  Male       65+  Not Reported          NaN   

   precipitation  humidity        date                                latlong  \
0            NaN       NaN  2007-01-01  ('37.6580958', '-111.09320889669256')   

    latitude   longitude                                            Weather  
0  37.658096 -111.093209  {'datetime': '2007-01-01', 'datetimeEpoch': 11...  


In [15]:
# Your API key
api_key = 'WTCKQMFXBWJSST6TGTZCVZF5Y' #spencerhamilton60@gmail.com
api_key = 'HR23S3BURZPNU7DBHQA9GAYZY' #rhamilt4@byu.edu
api_key = '3N2JVBHZHRS8SS4WVVA2DN7LW' #fake email 1
api_key = 'JAQ9AZB224ATD3UZ9YLR58USU' #fake email 2



# Display rows with existing weather data
existing_weather_data = df[df['Weather'].notna()]

# View the first few rows to inspect the structure and contents
#print(existing_weather_data.head())


existing_weather_data_length = df['Weather'].notna().sum()
#print(existing_weather_data_length)
# Save the updated dataset with relevant weather data
existing_weather_data.to_csv('weather__ed.csv', index=False)

In [ ]:
import requests
import time
import pandas as pd

# Assuming you have defined your base_url and api_key previously
base_url = 'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/'
api_key = api_key = '3N2JVBHZHRS8SS4WVVA2DN7LW' #fake email 1

def get_weather_data(location, date):
    url = f'{base_url}{location}/{date}?key={api_key}&contentType=csv'
    response = requests.get(url)
    
    if response.status_code == 200:
        # Extract the relevant data
        data = response.json()
        
        # Store the full weather data as a JSON string
        return data  # Return all data for now
    else:
        print(f"Error fetching data for {location} on {date}: {response.status_code}")
        return False  # Return None on error

# Loop through your DataFrame and fetch weather data
for i, row in df.iterrows():
    # Skip rows that already have weather data
    if pd.notna(row['Weather']):
        continue

    latitude = row['latitude']
    longitude = row['longitude']
    date = row['date']

    if (latitude is None) or (longitude is None):
        continue

    # Format location
    location = f"{latitude:.6f},{longitude:.6f}"
    
    # Request weather data
    weather_data = get_weather_data(location, date)
    
        

    if weather_data is not None:
        # Store the full JSON response in the 'Weather' column
        df.at[i, 'Weather'] = str(weather_data)  # Convert dict to string for storage
        
    else:
        break
        print("stopped bc it was bad")
    
    # Sleep to avoid hitting the API rate limit
    time.sleep(1)

# Save the updated dataset with relevant weather data
#df.to_csv('tester_yeah_with_weather.csv', index=False)

